# ARIMA&FFT

In [ ]:
import numpy as np
import pmdarima as pm
from scipy.fft import rfft, irfft
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
data = np.load('../../data/processed/labeled_data.npz')
electrical_values = data['electrical_values']
labels = data['labels']

## ARIMA

In [ ]:
arimas = [None] * len(electrical_values) # 507

for i, electrical_value in enumerate(electrical_values):
    arima_model = pm.auto_arima(electrical_value, start_p=1, d=1, start_q=1, max_p=2, max_d=1, max_q=2, seasonal=False, \
                                information_criterion='aicc')
    arimas[i] = arima_model

In [ ]:
nr_plots = 10
arimas_fitted = [arima.predict_in_sample() for arima in arimas]

for arima_fitted, electrical_value in zip(arimas_fitted[:nr_plots], electrical_values[:nr_plots]):
    arima_fitted[0] = electrical_value[0]
    arima_fitted[-1] = electrical_value[-1]
    plt.figure()
    plt.plot(electrical_value, 'b')
    plt.plot(arima_fitted, 'r')
    plt.show()

In [ ]:
for arima in arimas:
    print(arima.summary())

## FFT

In [ ]:
ffts = [None] * len(electrical_values) # 507

for i, electrical_value in enumerate(electrical_values):
    freqs = rfft(electrical_value)
    new_electrical_values = []
    for nr_vals in [1, 10, 100, 1000]:
        new_electrical_values.append(irfft(freqs[:nr_vals], n=len(electrical_value))) # 8760
    ffts[i] = new_electrical_values
ffts = np.array(ffts)
print(ffts[0], len(ffts[0]))

In [ ]:
nr_plots = 10

for fft, electrical_value in zip(ffts[:nr_plots], electrical_values[:nr_plots]):
    plt.figure()
    plt.plot(electrical_value)
    plt.show()
    plt.figure()
    f, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, sharex='col', sharey='row')
    ax0.plot(fft[0])
    ax1.plot(fft[1])
    ax2.plot(fft[2])
    ax3.plot(fft[3])
    plt.show()

In [ ]:
ffts[0]

In [ ]:
combined_data = [None] * len(electrical_values) # 507

for i, (electrical_value, arima_fitted, fft) in enumerate(zip(electrical_values, arimas_fitted, ffts)):
    electrical_value = np.expand_dims(electrical_value, 0)
    arima_fitted = np.expand_dims(arima_fitted, 0)
    data_row = np.concatenate((electrical_value, arima_fitted, fft), 0)
    combined_data[i] = data_row

combined_data = np.array(combined_data)
print(combined_data[:10])

In [ ]:
np.savez_compressed('../../data/processed/combined_data.npz', combined_data=combined_data, labels=labels)